In [1]:
import pandas as pd
import mysql
import mysql.connector

In [2]:
queries_creacion = [] #Lista con las queries para crear cada tabla
query_crear_tabla_ofertas = """
CREATE TABLE ofertas (
    id_oferta VARCHAR(255) PRIMARY KEY,
    empresa VARCHAR(255),
    titulo VARCHAR(255),
    ciudad VARCHAR(255),
    salario_desde NUMERIC,
    salario_hasta NUMERIC,
    variabe NUMERIC,
    vacaciones NUMERIC,
    teletrabajo ENUM("Remoto", "Hibrido", "Presencial"),
    jornada_tipo VARCHAR(255),
    turno_continuo BOOLEAN NULL,
    imprescindible_residir ENUM("País Puesto", "No requerido", "Provincia Puesto") NULL,
    funciones VARCHAR(255),
    contrato_tipo VARCHAR(255),
    nivel_profesional VARCHAR(255),
    formacion_minima VARCHAR(255),
    personas_a_cargo TINYINT UNSIGNED NULL,
    experiencia VARCHAR (255),
    vacantes TINYINT UNSIGNED NULL,
    cvs_inscritos NUMERIC,
    oferta_activada BOOLEAN,
    fecha_publicacion DATETIME NULL,
    timestamp DATETIME NULL,
    url TEXT
);
"""
queries_creacion.append(query_crear_tabla_ofertas)

In [3]:
query_crear_tabla_tecnologias = """
CREATE TABLE tecnologias (
    tecnologia VARCHAR (255),
    tec_id INT UNSIGNED PRIMARY KEY
);
"""
queries_creacion.append(query_crear_tabla_tecnologias)

In [4]:
query_crear_tabla_tecnologias_relacion = """
CREATE TABLE tecnologias_relacion(
    id_oferta VARCHAR(255),
    tec_id INT UNSIGNED,
    FOREIGN KEY (id_oferta) REFERENCES ofertas(id_oferta),
    FOREIGN KEY (tec_id) REFERENCES tecnologias(tec_id)
);
"""
queries_creacion.append(query_crear_tabla_tecnologias_relacion)

In [5]:
query_crear_tabla_habilidades = """
CREATE TABLE habilidades (
    habilidad VARCHAR (255),
    hab_id INT UNSIGNED PRIMARY KEY
);
"""
queries_creacion.append(query_crear_tabla_habilidades)

In [6]:
query_crear_tabla_habilidades_relacion = """
CREATE TABLE habilidades_relacion(
    id_oferta VARCHAR(255),
    hab_id INT UNSIGNED,
    FOREIGN KEY (id_oferta) REFERENCES ofertas(id_oferta),
    FOREIGN KEY (hab_id) REFERENCES habilidades(hab_id)
);
"""
queries_creacion.append(query_crear_tabla_habilidades_relacion)

In [7]:
query_crear_tabla_idiomas = """
CREATE TABLE idiomas (
    id_oferta VARCHAR(255),
    idioma VARCHAR (30),
    nivel ENUM("Bajo", "Medio", "Alto", "Excelente") NULL,
    FOREIGN KEY (id_oferta) REFERENCES ofertas(id_oferta)

);
"""
queries_creacion.append(query_crear_tabla_idiomas)

In [ ]:
query_crear_tabla_ciudades_coordenadas = """
CREATE TABLE ciudades_coordenadas (
    nombre VARCHAR(100),
    latitud DECIMAL(9, 6),
    longitud DECIMAL(9, 6)
);
"""
queries_creacion.append(query_crear_tabla_ciudades_coordenadas)

In [8]:
database = "ofertas_de_empleo"
db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "Scarface",
                             database = None)
cursor = db.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
cursor.close()
db.close()

In [9]:
for query in queries_creacion:
    database = "ofertas_de_empleo"
    db = mysql.connector.connect(host     = "localhost",
                                 user     = "root",
                                 password = "Scarface",
                                 database = database)    
    cursor = db.cursor()
    cursor.execute(query)
    cursor.fetchall()
    cursor.close()
    db.close()    

In [10]:
df_tecnologias = pd.read_csv("/home/bosser/Escritorio/proyectofinal/tecnologias_final.csv")
df_tecnologias_relacion = pd.read_csv("/home/bosser/Escritorio/proyectofinal/tecnologias_relacion_final.csv")
df_habilidades = pd.read_csv("/home/bosser/Escritorio/proyectofinal/habilidades_final.csv")
df_habilidades_relacion = pd.read_csv("/home/bosser/Escritorio/proyectofinal/habilidades_relacion_final.csv")
df_ofertas = pd.read_csv("/home/bosser/Escritorio/proyectofinal/ofertas_final.csv")
df_idiomas = pd.read_csv("/home/bosser/Escritorio/proyectofinal/idiomas.csv")
###########df_ciudades + ciudades_coordenadas

In [11]:
tablas = ["ofertas", "tecnologias", "tecnologias_relacion", "habilidades", "habilidades_relacion", "idiomas,", "ciudades"]
dfs = [df_ofertas, df_tecnologias, df_tecnologias_relacion, df_habilidades, df_habilidades_relacion, df_idiomas #,df_ciudades
      ]


In [12]:


for tabla, df in zip(tablas, dfs):
    database = "ofertas_de_empleo"
    
    db = mysql.connector.connect(host     = "localhost",
                                 user     = "root",
                                 password = "Scarface",
                                 database = database)
    
    cursor = db.cursor()
    
    # Seleccionamos las columnas de la tabla, omitiendo la Primary Key
    cursor.execute(f"SELECT * FROM {tabla} LIMIT 0;")
    column_names = cursor.column_names
    cursor.fetchall()
    
    
    insert_query = f"INSERT INTO {tabla} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})".replace("'", "")
    values = [tuple(row) for row in df.values] # Aquí "casteamos" cada fila en el df para que sea una tupla
    
    # .executemany ejecuta el query de INSERT INTO con cada uno de los elementos de "values"
    cursor.executemany(insert_query, values)
    
    # Guarda los resultados
    db.commit()
    
    print(f"Añadidas: {cursor.rowcount} filas")
    
    cursor.fetchall() # Vaciamos el cursor
    cursor.close()
    db.close()

Añadidas: 4830 filas
Añadidas: 1948 filas
Añadidas: 16214 filas
Añadidas: 65 filas
Añadidas: 12170 filas
Añadidas: 3645 filas
